In [ ]:
import torch
import torchvision
from input_pipeline import ImageLoader
from evaluation_metrics import calc_evaluation_metrics, eval_scores, print_eval_scores
from input_pipeline import create_dataset_split
import numpy as np
PATH_TO_IMAGES = 'data/img_align_celeba'
PATH_TO_LABELS = 'data/list_attr_celeba.csv'

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', weights=torchvision.models.MobileNet_V2_Weights.IMAGENET1K_V2)

dataset = ImageLoader(PATH_TO_IMAGES, PATH_TO_LABELS)
train_data, val_data, test_data = create_dataset_split(dataset=dataset, batch_size=64)

In [ ]:
train_img = next(iter(test_data))[0]
print(min(train_img[2].flatten()))
print(max(train_img[2].flatten()))

val_img = next(iter(val_data))[0]
print(min(val_img[2].flatten()))
print(max(val_img[2].flatten()))

In [31]:
classifier = torch.nn.Sequential(
    torch.nn.Dropout(0.3),
    torch.nn.Linear(1280, 128),
    torch.nn.BatchNorm1d(128),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(128, 64),
    torch.nn.BatchNorm1d(64),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.1),
    torch.nn.Linear(64, 40)
)
model.classifier = classifier
print(model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [ ]:
for layer in model.classifier.children():
    if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

In [ ]:
def validate(model, val_data):
    model.eval()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    avg_partial_accuracy = avg_f1_score = avg_recall_score = avg_precision_score = avg_hamming_loss = avg_hamming_score = 0.0
    avg_label_wise_accuracy_score = np.zeros(shape=(40))

    NUM_OF_BATCHES = len(val_data)
    
    for i, batch in enumerate(val_data):
        with torch.no_grad():
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            result = outputs > 0.5

            f1_score, recall, precision, hamming_loss, ham_score, partial_accuracy, label_wise_accuracy =\
                        eval_scores(labels.cpu().detach().numpy(), result.cpu().detach().numpy(), print_out=True, batch=i)
            
            avg_f1_score += f1_score
            avg_recall_score += recall
            avg_precision_score += precision
            avg_hamming_loss += hamming_loss
            avg_hamming_score += ham_score
            avg_partial_accuracy += partial_accuracy
            avg_label_wise_accuracy_score += label_wise_accuracy
        
    avg_f1_score /= NUM_OF_BATCHES
    avg_recall_score /= NUM_OF_BATCHES
    avg_precision_score /= NUM_OF_BATCHES
    avg_hamming_loss /= NUM_OF_BATCHES
    avg_hamming_score /= NUM_OF_BATCHES
    avg_partial_accuracy /= NUM_OF_BATCHES
    avg_label_wise_accuracy_score /= NUM_OF_BATCHES

    print_eval_scores(avg_f1_score, avg_recall_score, avg_precision_score, avg_hamming_loss, avg_hamming_score, avg_partial_accuracy, avg_label_wise_accuracy_score)


In [ ]:
def fit(model, train_data, val_data, optimizer, loss_fn, epochs):
        

        # Frozen feature layers
        for param in model.features.parameters():
            param.requires_grad = False

        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        for epoch in range(epochs):
            model.train(mode=True)
            avg_partial_accuracy = avg_f1_score = avg_recall_score = avg_precision_score = avg_hamming_loss = avg_hamming_score = 0.0
            avg_label_wise_accuracy_score = np.zeros(shape=(40))

            for i, batch in enumerate(train_data):
                images, labels = batch
                images = images.to(device)
                labels = labels.to(device)

                # Threshold fol binary classification
                theta = 0.5

                optimizer.zero_grad()
                outputs = model(images)
                loss = loss_fn(outputs, labels)
                loss.backward()
                optimizer.step()
                
                result = outputs > theta
                
                # Measure model performance for every batch
                f1_score, recall, precision, hamming_loss, ham_score, partial_accuracy, label_wise_accuracy =\
                      eval_scores(labels.cpu().detach().numpy(), result.cpu().detach().numpy(), print_out=True, epoch=epoch, batch=i)
            
            # Evaluetion metrics for every epoch
                
            validate(model, val_data)

In [30]:
loss = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

fit(model, train_data, val_data, optimizer, loss, 1)
#validate(model, val_data)


========================Epoch: 1, Batch: 1========================
F1 score:
0.18062403371706443
Recall:
0.1572854229104229
Precision:
0.27857931998557
Hamming loss:
0.2828125
Hamming score:
0.10614924157155195
Partial accuracy:
0.7171875
Label wise accuracy:
[0.765625 0.765625 0.515625 0.609375 0.9375   0.859375 0.546875 0.640625
 0.53125  0.8125   0.921875 0.75     0.921875 0.875    0.40625  0.6875
 0.859375 0.9375   0.671875 0.5      0.6875   0.5625   0.921875 0.75
 0.265625 0.734375 0.765625 0.5625   0.828125 0.890625 0.859375 0.515625
 0.78125  0.6875   0.734375 0.9375   0.546875 0.90625  0.90625  0.328125]



========================Epoch: 1, Batch: 2========================
F1 score:
0.21438040887411597
Recall:
0.1824822313103563
Precision:
0.3120597718253968
Hamming loss:
0.253125
Hamming score:
0.1268550000856986
Partial accuracy:
0.746875
Label wise accuracy:
[0.796875 0.71875  0.671875 0.765625 0.9375   0.859375 0.671875 0.84375
 0.625    0.875    0.90625  0.71875  0.875   

TypeError: can only concatenate str (not "int") to str